In [1]:
#Scrape Products From Amazon

In [2]:
import pandas as pd 
from selenium import webdriver
from bs4 import BeautifulSoup

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get('https://www.amazon.in/s?k=mobiles&crid=2944PM084A0K4&sprefix=mobiles%2Caps%2C271&ref=nb_sb_noss_1')

In [5]:
html_data = BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
No_of_pages = int(html_data.find('span',{'class':'s-pagination-item s-pagination-disabled'}).text)

In [7]:
No_of_pages

20

In [8]:
products = html_data.find_all('div',{'data-component-type':'s-search-result'})

In [9]:
import time

In [27]:
from tqdm import tqdm  # Ensure tqdm is imported

titles = []
images = []
ratings = []
prices = []
for i in tqdm(range(No_of_pages)):
    url = 'https://www.amazon.in/s?k=laptops&crid=2KEFMO61KR1T7&sprefix=laptop%2Caps%2C232&ref=nb_sb_noss_2&page='+str(i+1)
    driver.get(url)
    time.sleep(3)
    html_data = BeautifulSoup(driver.page_source, 'html.parser')
    products = html_data.find_all('div',{'data-component-type':'s-search-result'})
    for product in products:
        title = product.find('span',{'class':'a-size-medium a-color-base a-text-normal'}).text
        if title == None:
            break
        titles.append(title)
        img = product.find('img')['src']
        images.append(img)
        rating = product.find('span',{'class':'a-icon-alt'})
        if rating  == None:
            rating = 'No Rating'
        else:
            rating = product.find('span',{'class':'a-icon-alt'}).text
        ratings.append(rating)
        
        price = product.find('span',{'class':'a-price-whole'})
        if price == None:
            price = 'Not Available'
        else:
            prices.append('₹' + price.text)

100%|██████████| 20/20 [01:52<00:00,  5.63s/it]


In [28]:
len(titles)

426

In [29]:
min_length = min(len(titles), len(images), len(ratings), len(prices))
titles = titles[:min_length]
images = images[:min_length]
ratings = ratings[:min_length]
prices = prices[:min_length]

data = pd.DataFrame({'titles': titles, 'images': images, 'ratings': ratings, 'prices': prices})


In [35]:
data

,titles,images,ratings,prices
0,"Dell 15 Thin & Light Laptop, Intel Core i5-123...",https://m.media-amazon.com/images/I/81T5jRKDkv...,3.6 out of 5 stars,"₹52,990"
1,Lenovo IdeaPad Pro 5 Intel Evo Core Ultra 9 18...,https://m.media-amazon.com/images/I/618ac7UaMr...,4.5 out of 5 stars,"₹1,09,990"
2,(Refurbished) Lenovo ThinkPad 6th Gen Intel Co...,https://m.media-amazon.com/images/I/61vGumezio...,3.3 out of 5 stars,"₹15,430"
3,Acer Aspire Lite 12th Gen Intel Core i3-1215U ...,https://m.media-amazon.com/images/I/71czGb00k5...,4.0 out of 5 stars,"₹29,990"
4,(Refurbished) Lenovo ThinkPad 5th Gen Intel Co...,https://m.media-amazon.com/images/I/617Ykb5YvL...,3.7 out of 5 stars,"₹12,980"
...,...,...,...,...
420,"HP Laptop 15, Intel® Core i5-1334U,15.6-inch(3...",https://m.media-amazon.com/images/I/51GMd4x98Q...,3.9 out of 5 stars,"₹1,04,990"
421,"ASUS ROG Zephyrus G14, 14-inch (35.56 cms) WQH...",https://m.media-amazon.com/images/I/81Nwu5Hp0j...,4.0 out of 5 stars,"₹72,990"
422,(Renewed) Dell M6800 Precision Hybrid 17 Inche...,https://m.media-amazon.com/images/I/61BVsCGR9+...,No Rating,"₹72,990"
423,(Renewed) Dell M6800 Precision 17 Inches Scree...,https://m.media-amazon.com/images/I/51k7VNMXMy...,No Rating,"₹1,09,990"


In [31]:
data.to_csv('Laptop Products.csv')